In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Substance_Use_Disorder"
cohort = "GSE148375"

# Input paths
in_trait_dir = "../../input/GEO/Substance_Use_Disorder"
in_cohort_dir = "../../input/GEO/Substance_Use_Disorder/GSE148375"

# Output paths
out_data_file = "../../output/preprocess/Substance_Use_Disorder/GSE148375.csv"
out_gene_data_file = "../../output/preprocess/Substance_Use_Disorder/gene_data/GSE148375.csv"
out_clinical_data_file = "../../output/preprocess/Substance_Use_Disorder/clinical_data/GSE148375.csv"
json_path = "../../output/preprocess/Substance_Use_Disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"An Exome-Wide Association Study Identifies New Susceptibility Loci for the Risk of Nicotine Dependence in African-American Populations"
!Series_summary	"Cigarette smoking is one of the largest causes of preventable death worldwide. Smoking behaviors, including age at smoking initiation (ASI), smoking dependence (SD), and smoking cessation (SC), are all complex phenotypes determined by both genetic and environmental factors as well as their interactions. To identify susceptibility loci for each smoking phenotype, numerous studies have been conducted, with approaches including genome-wide linkage scans, candidate gene-based association analysis, and genome-wide association study (GWAS). Therefore, we conducted an exome-wide association study to identify new susceptibility loci for the risk of nicotine dependence in African-American populations."
!Series_overall_design	"To reveal the molecular mechanism underling each smoking phenotype, we used high-

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From background information, this dataset seems to be a genetic association study focusing on variants, 
# not gene expression. It specifically mentions "exome-wide association study" and analyzing genetic variants.
is_gene_available = False

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the sample characteristics dictionary:

# Trait (Substance Use Disorder - specifically nicotine dependence in this case)
# Key 6 contains 'smoking_status' which can be used as our trait variable
trait_row = 6  

# Age data is available in key 1
age_row = 1

# Gender data is available in key 2
gender_row = 2

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert smoking status to binary trait values."""
    if not isinstance(value, str):
        return None
    # Extract value part after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value == 'Smoker':
        return 1  # Has nicotine dependence
    elif value in ['Non-smoker', 'Ex-smoker']:
        return 0  # Does not have current nicotine dependence
    else:
        return None
        
def convert_age(value):
    """Convert age to numerical continuous values."""
    if not isinstance(value, str):
        return None
    # Extract value part after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        # Skip placeholder values like -9
        if value == '-9':
            return None
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary values (0 for female, 1 for male)."""
    if not isinstance(value, str):
        return None
    # Extract value part after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'male':
        return 1
    elif value.lower() == 'female':
        return 0
    else:
        return None

# 3. Save Metadata - Initial Filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, clinical data is available, so we extract features
if trait_row is not None:
    # We need clinical_data from previous steps to run this part
    # Assuming clinical_data is available from previous steps
    try:
        # Create output directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Extract selected clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        preview_result = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview_result)
        
        # Save to CSV
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except NameError:
        print("Cannot extract clinical features: clinical_data is not available.")
else:
    print("Clinical data is not available for this cohort.")

Preview of selected clinical features:
{'GSM4463148': [1.0, 39.0, 1.0], 'GSM4463149': [1.0, 42.0, 1.0], 'GSM4463150': [1.0, 32.0, 0.0], 'GSM4463151': [1.0, 33.0, 1.0], 'GSM4463152': [1.0, 48.0, 0.0], 'GSM4463153': [1.0, 33.0, 1.0], 'GSM4463154': [1.0, 29.0, 0.0], 'GSM4463155': [0.0, 46.0, 0.0], 'GSM4463156': [1.0, 46.0, 0.0], 'GSM4463157': [1.0, 53.0, 0.0], 'GSM4463158': [1.0, 24.0, 1.0], 'GSM4463159': [1.0, 50.0, 1.0], 'GSM4463160': [1.0, 42.0, 0.0], 'GSM4463161': [1.0, 39.0, 0.0], 'GSM4463162': [1.0, 27.0, 1.0], 'GSM4463163': [0.0, 24.0, 0.0], 'GSM4463164': [1.0, 17.0, 1.0], 'GSM4463165': [0.0, 16.0, 0.0], 'GSM4463166': [1.0, 19.0, 1.0], 'GSM4463167': [1.0, 42.0, 1.0], 'GSM4463168': [1.0, 61.0, 1.0], 'GSM4463169': [1.0, 42.0, 1.0], 'GSM4463170': [1.0, 46.0, 0.0], 'GSM4463171': [1.0, 53.0, 1.0], 'GSM4463172': [1.0, 37.0, 0.0], 'GSM4463173': [1.0, 38.0, 1.0], 'GSM4463174': [1.0, 35.0, 0.0], 'GSM4463175': [1.0, 29.0, 0.0], 'GSM4463176': [1.0, 24.0, 0.0], 'GSM4463177': [1.0, 52.0, 1.0], 